# Information Retrieval (NLP Project)

From Chapter 3 of Getting Started with Natural Language Processing (2022, Kochmar)

Data from Centre for Inventions and Scientific Infromation (CISI) at http://ir.dcs.gla.ac.uk/resources/test_collections/cisi/

The goal of this project is to create an algorithm that will return a set of documents, in relevance order, to a query. 

# 1. Preliminary Steps

In [1]:
# import libraries
import nltk
import string # has .punctuation list of punctuation
import math
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from operator import itemgetter # helps python sort dictionaries by keys or values

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yang0108\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yang0108\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 2. Create and populate data structures

We need three data structures to start with.

First, we need the documents to be searched. We will keep these in a dictionary with the keys as unique document IDs, and the values as a string of the document's content.

We also need the queries that we will train and test our algorithm on. We will likewise keep these in a table with the keys as unique query IDs, and the values as a string of the query.

Last, we need a mapping of the queries to the documents that should be returned for each query, so our model can be trained on these gold standard mappings. We will keep the mappings in a table with the keys as the query IDs, and the values as lists of document IDs that match each query.

## 2.1 Documents

In [2]:
# function to populate documents dict
def read_documents():
    f = open("Data/cisi/CISI.ALL")
    
    # keep the result of merging the field identifier (.I, .T, .A, .W, .X)
    # for each new .I entry with its content in a string
    merged = ""
    
    # update merged string in for loop
    for a_line in f.readlines():
        
        # if line starts with .
        if a_line.startswith("."):
            
            # append new line to merged and the next line of text,
            # stripped of before and after whitespace
            merged += "\n" + a_line.strip()
        
        else:
            
            # append one space and next line of text
            merged += " " + a_line.strip()
    
    # initialize empty dictionary
    documents = {}

    # keep content (val) and id (key) in each dictionary entry
    content = ""
    doc_id = ""
    
    # iterate through all lines (split with \n) in string merged
    for a_line in merged.split("\n"):
        
        # extract doc_id from line with the .I field identifier
        if a_line.startswith(".I"):
            doc_id = a_line.split(" ")[1].strip()
            
        # stop collecting doc_id and content if line starts with .X
        elif a_line.startswith(".X"):
            
            # store current content and doc_id in documents dict
            documents[doc_id] = content
            
            # re-initialize empty strings for content and doc_id
            content = ""
            doc_id = ""
            
        else:
            
            # remove first 3 chars (field identifiers) and append text
            # to content string with space at end
            content += a_line.strip()[3:] + " "
            
    f.close()
    
    return documents

# read in data (call function)
documents = read_documents()

In [3]:
# check number of documents (should be 1460)
print(len(documents))

# check text of first document (should be about the dewey decimal system)
print(documents.get("1"))

1460
 18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad. 


## 2.2 Queries

In [4]:
# function to populate queries dict
def read_queries():
    f = open("Data/cisi/CISI.QRY")
    
    # merge each entry (starts with .I) in string
    merged = ""
    
    # iterate through all lines
    for a_line in f.readlines():
        
        # if line starts with .
        if a_line.startswith("."):
            
            # append new line to merged, and the line of text
            # stripped of any before and after whitespace
            merged += "\n" + a_line.strip()
            
        else:
            
            # else append one space and then line of text
            merged += " " + a_line.strip()
    
    # initialize empty queries dictionary
    queries = {}

    # initialize strings for content and query_id
    content = ""
    qry_id = ""

    # iterate through lines in merged string, split by new lines
    for a_line in merged.split("\n"):
        
        # if the line starts with .I
        if a_line.startswith(".I"):
            
            # if content string is not empty
            if not content=="":
                
                # store current content and qry_id to queries dictionary
                queries[qry_id] = content
                
                # initialize new empty strings for content and qry_id
                content = ""
                qry_id = ""
            
            # split the line by space and store second element as qry_id
            qry_id = a_line.split(" ")[1].strip()
            
        # else if the line starts with .W or .T    
        elif a_line.startswith(".W") or a_line.startswith(".T"):
            
            # append line (from fourth char) to current content
            content += a_line.strip()[3:] + " "
    
    # extra step to add entry for last query to dictionary
    queries[qry_id] = content
    
    f.close()
    
    return queries

# read in data (call function)
queries = read_queries()

In [5]:
# check number of queries (should be 112)
print(len(queries))

# check text of first query (should be about descriptive titles)
print(queries.get("1"))

112
What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles? 


## 2.3 Mappings

In [6]:
# function to populate mappings dict
def read_mappings():
    f = open("Data/cisi/CISI.REL")
    
    # initialize empty dictionary
    mappings = {}
    
    # iterate over lines in file
    for a_line in f.readlines():
        
        # split each line into columns
        voc = a_line.strip().split()
        
        # access the first column, strip, and assign to key
        key = voc[0].strip()
        
        # access second column, strip, and assign to current_value
        current_value = voc[1].strip()
        
        # initialize empty list for values
        value = []
        
        # if key is already in mappings
        if key in mappings.keys():
            
            # assign value to stored list of values
            value = mappings.get(key)
        
        # append current_value to list of values
        value.append(current_value)
        
        # assign list of values to key in dict
        mappings[key] = value

    f.close()
    
    return mappings

# read in data (call function)
mappings = read_mappings()

In [7]:
# check length of dict (should be 76)
print(len(mappings))

# check keys in dict
print(mappings.keys())

# check documents associated with qry_id 1
# (should be list of 46 document ids, starting
# with 28 and ending with 1281)
print(mappings.get("1"))

76
dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '37', '39', '41', '42', '43', '44', '45', '46', '49', '50', '52', '54', '55', '56', '57', '58', '61', '62', '65', '66', '67', '69', '71', '76', '79', '81', '82', '84', '90', '92', '95', '96', '97', '98', '99', '100', '101', '102', '104', '109', '111'])
['28', '35', '38', '42', '43', '52', '65', '76', '86', '150', '189', '192', '193', '195', '215', '269', '291', '320', '429', '465', '466', '482', '483', '510', '524', '541', '576', '582', '589', '603', '650', '680', '711', '722', '726', '783', '813', '820', '868', '869', '894', '1162', '1164', '1195', '1196', '1281']


# 3. Practice simple Boolean search algorithms

These algorithms will not be helpful in practice because they are too broad and too narrow, respectively, to help find relevant matching documents to a query. But, we will keep them here as practice.

Steps:
1. Extract words from query
2. For each document, compare words in document to words in query
3. Return document as relevant (True) if any of query words occur in document

## 3.1 Simple text preprocessing

This first attempt at text processing does the following:
   
   - makes all letters lowercase
   - tokenizes strings into words with NLTK's word_tokenizer

In [8]:
# define function to make text lowercase and tokenize
# return list of words in text
def get_words(text):
    word_list = [word for word in word_tokenize(text.lower())]
    return word_list

In [9]:
# initialize empty dictionaries for words in documents and queries
doc_words = {}
qry_words = {}

# loop over document ids in documents dictionary
for doc_id in documents.keys():
    
    # call get_words function on value text for doc_id
    # assign it to key doc_id in doc_words dict
    doc_words[doc_id] = get_words(documents.get(doc_id))

# same for queries
for qry_id in queries.keys():
    qry_words[qry_id] = get_words(queries.get(qry_id))

In [10]:
# check number of documents in doc_words (should be 1460)
print(len(doc_words))

# check words associated with doc_id 1 (should be list of
# words about dewey decimal system)
print(doc_words.get("1"))

# check number of words in list (should be 113)
print(len(doc_words.get("1")))

1460
['18', 'editions', 'of', 'the', 'dewey', 'decimal', 'classifications', 'comaromi', ',', 'j.p.', 'the', 'present', 'study', 'is', 'a', 'history', 'of', 'the', 'dewey', 'decimal', 'classification', '.', 'the', 'first', 'edition', 'of', 'the', 'ddc', 'was', 'published', 'in', '1876', ',', 'the', 'eighteenth', 'edition', 'in', '1971', ',', 'and', 'future', 'editions', 'will', 'continue', 'to', 'appear', 'as', 'needed', '.', 'in', 'spite', 'of', 'the', 'ddc', "'s", 'long', 'and', 'healthy', 'life', ',', 'however', ',', 'its', 'full', 'story', 'has', 'never', 'been', 'told', '.', 'there', 'have', 'been', 'biographies', 'of', 'dewey', 'that', 'briefly', 'describe', 'his', 'system', ',', 'but', 'this', 'is', 'the', 'first', 'attempt', 'to', 'provide', 'a', 'detailed', 'history', 'of', 'the', 'work', 'that', 'more', 'than', 'any', 'other', 'has', 'spurred', 'the', 'growth', 'of', 'librarianship', 'in', 'this', 'country', 'and', 'abroad', '.']
113


In [11]:
# check number of queries in qry_words (should be 112)
print(len(qry_words))

# check words associated with qry_id 1 (should be list of
# words about descriptive titles)
print(qry_words.get("1"))

# check number of words in list (should be 38)
print(len(qry_words.get("1")))

112
['what', 'problems', 'and', 'concerns', 'are', 'there', 'in', 'making', 'up', 'descriptive', 'titles', '?', 'what', 'difficulties', 'are', 'involved', 'in', 'automatically', 'retrieving', 'articles', 'from', 'approximate', 'titles', '?', 'what', 'is', 'the', 'usual', 'relevance', 'of', 'the', 'content', 'of', 'articles', 'to', 'their', 'titles', '?']
38


## 3.2 Algorithm 1: Too Broad

This preliminary algorithm returns all documents that match ANY word in a query. In practice, this is too broad because almost all queries and documents will have at least some of the same words, especially considering common words like "the", "and", "is", etc. 

When we run this algorithm on one query in our training data, 1400 out of a possible 1460 documents are returned.

In [12]:
# define function that looks for query words in documents
def retrieve_documents(doc_words, query):
    
    # initialize empty list to store docs found
    docs = []
    
    # iterate through all doc_ids in doc_words
    for doc_id in doc_words.keys():
        
        # set found flag to False (will turn to True if any
        # matching words are found)
        found = False
        
        # initialize index of query word in query word list
        i = 0
        
        # while no matches have been found (while loop)
        # two conditions to keep searching: 
        # 1) haven't reached end of query
        # 2) found flag is false
        while i < len(query) and not found:
            
            # set word to ith element of query
            word = query[i]
            
            # check if word is in document
            if word in doc_words.get(doc_id):
                
                # append doc_id to docs list
                docs.append(doc_id)
                
                # set found flag to True
                found = True
            
            else:
                
                # increment i to see next word
                i += 1
        
    # return docs list of docs that contain at least 1 matching word to query
    return docs
            

In [13]:
# check query "3"
docs = retrieve_documents(doc_words, qry_words.get("3"))

# print out docs (up to 100 results)
print(docs[:100])

# print length of docs (should be 1400)
print(len(docs))

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102']
1400


## 3.3 Algorithm 2: Too Narrow

This algorithm returns documents that match ALL words in a query. This is too narrow in scope, and does not fit the task; in practice, we don't care if our query (sometimes a question that starts with "what" or "which") has some words that don't appear in the documents (in this case, abstracts, which may not have any questions in them and therefore are not likely to include such words). 

In a check on one query, the algorithm returned zero documents that matched all the words in the query.

In [14]:
# define function that looks for query words in documents
def retrieve_documents(doc_words, query):
    
    # initialize empty list to store docs found
    docs = []
    
    # iterate through all doc_ids in doc_words
    for doc_id in doc_words.keys():
        
        # set found flag to True
        found = True
        
        # initialize index of query word in query word list
        i = 0
        
        # while no matches have been found (while loop)
        # condition to keep searching: haven't reached end of query
        while i < len(query) and found:
            
            # set word to ith element of query
            word = query[i]
            
            # check if word is NOT in document
            if not word in doc_words.get(doc_id):
                
                # if word is not in doc, turn found to False and move on
                found = False
                    
            else:
                        
                # otherwise, go onto the next word in query
                i += 1
            
        # if all words are found in document (i.e. i equals the index 
        # of the last element in query)
        if i == len(query) - 1:
                
            # append the doc_id to docs list
            docs.append(doc_id)
        
    # return docs list of docs that contain all words in query
    return docs
            

In [15]:
# check query "112"
docs = retrieve_documents(doc_words, qry_words.get("112"))

# print out docs (up to 100 results) (should have no matching docs)
print(docs[:100])

# print length of docs (0)
print(len(docs))

[]
0


# 4. Text preprocessing (revised)

In addition to the preprocessing steps in the last section, we need more in order to create a helpful information retrieval algorithm. Here, we introduce a text processing function that does the following:
   - makes all letters lowercase
   - tokenizes strings into words with NLTK's word_tokenizer
   - removes stopwords (from NLTK's stopwords list)
   - removes punctuation (from Python's string module punctuation list)
   - stems the words into terms (using NLTK's Lancaster Stemmer)

## 4.1 Removing stopwords

In [16]:
def process(text):
    
    # set set of stopwords
    stoplist = set(stopwords.words('english'))
    
    # set list of words in text that are lowercase, not
    # stopwords, and not punctuation marks
    word_list = [word for word in word_tokenize(text.lower())
                if not word in stoplist and
                not word in string.punctuation]
    
    return word_list

In [17]:
# check process function on document 1
word_list = process(documents.get("1"))
print(word_list)

['18', 'editions', 'dewey', 'decimal', 'classifications', 'comaromi', 'j.p.', 'present', 'study', 'history', 'dewey', 'decimal', 'classification', 'first', 'edition', 'ddc', 'published', '1876', 'eighteenth', 'edition', '1971', 'future', 'editions', 'continue', 'appear', 'needed', 'spite', 'ddc', "'s", 'long', 'healthy', 'life', 'however', 'full', 'story', 'never', 'told', 'biographies', 'dewey', 'briefly', 'describe', 'system', 'first', 'attempt', 'provide', 'detailed', 'history', 'work', 'spurred', 'growth', 'librarianship', 'country', 'abroad']


## 4.2 Stemming

In [18]:
def process(text):
    
    # set set of stopwords
    stoplist = set(stopwords.words('english'))
    
    # instantiate stemmer
    st = LancasterStemmer()
    
    # set list of words in text that are lowercase, not
    # stopwords, and not punctuation marks
    word_list = [st.stem(word) for word in word_tokenize(text.lower())
                if not word in stoplist and
                not word in string.punctuation]
    
    return word_list

In [19]:
# check process function on document 27 (should return list starting
# with 'cost', 'analys', 'sim', 'proc')
word_list = process(documents.get("27"))
print(word_list)

['cost', 'analys', 'sim', 'proc', 'evalu', 'larg', 'inform', 'system', 'bourn', 'c.p', 'ford', 'd.f', 'comput', 'program', 'writ', 'us', 'sim', 'several-year', 'op', 'inform', 'system', 'comput', 'estim', 'expect', 'op', 'cost', 'wel', 'amount', 'equip', 'personnel', 'requir', 'tim', 'period', 'program', 'us', 'analys', 'sev', 'larg', 'system', 'prov', 'us', 'research', 'tool', 'study', 'system', 'many', 'compon', 'interrel', 'op', 'equ', 'man', 'analys', 'would', 'extrem', 'cumbersom', 'tim', 'consum', 'perhap', 'ev', 'impract', 'pap', 'describ', 'program', 'show', 'exampl', 'result', 'sim', 'two', 'sev', 'suggest', 'design', 'spec', 'inform', 'system']


In [20]:
# check process function on a string of words
word_list = process("organize, organizing, organizational, organ, organic, organizer")
print(word_list)

['org', 'org', 'org', 'org', 'org', 'org']


# 5. Information Weighing

So far we have focused on processing text and returning documents to queries that match some or all words in the query, but even if we were able to get a reasonable balance in how many words to match, we would still only receive a list of matching documents that are not in order in terms of their relevance to the query. In practice, it is very helpful to return documents that match the query the "most" (however defined) first, and then descend in relevance from there.

Towards that end, we need to consider term frequencies in the documents and queries, as we implement below.

## 5.1 Term frequencies

In [21]:
# modify preprocessing function to return dictionary of
# terms in text and their counts
# change name of preprocessing function from 'process' to 'get_terms'
def get_terms(text):
    stoplist = set(stopwords.words('english'))
    
    # initialize empty terms dictionary
    terms = {}
    
    st = LancasterStemmer()
    word_list = [st.stem(word) for word in word_tokenize(text.lower())
                if not word in stoplist and not word in string.punctuation]
    
    # iterate over words in word_list
    for word in word_list:
        
        # get the current value of word in terms dictionary
        # and increment by one
        terms[word] = terms.get(word, 0) + 1
    
    return terms

In [22]:
# make empty dictionaries to store document terms and query terms
doc_terms = {}
qry_terms = {}

# populate dictionaries with get_terms function
for doc_id in documents.keys():
    doc_terms[doc_id] = get_terms(documents.get(doc_id))
for qry_id in queries.keys():
    qry_terms[qry_id] = get_terms(queries.get(qry_id))

In [23]:
# check number of elements in doc_terms (should be 1460)
print(len(doc_terms))

# check terms for first element in doc_terms
print(doc_terms.get("1"))

# check how many unique terms are in the first element of doc_terms (should be 43)
print(len(doc_terms.get("1")))

1460
{'18': 1, 'edit': 4, 'dewey': 3, 'decim': 2, 'class': 2, 'comarom': 1, 'j.p.': 1, 'pres': 1, 'study': 1, 'hist': 2, 'first': 2, 'ddc': 2, 'publ': 1, '1876': 1, 'eighteen': 1, '1971': 1, 'fut': 1, 'continu': 1, 'appear': 1, 'nee': 1, 'spit': 1, "'s": 1, 'long': 1, 'healthy': 1, 'lif': 1, 'howev': 1, 'ful': 1, 'story': 1, 'nev': 1, 'told': 1, 'biograph': 1, 'brief': 1, 'describ': 1, 'system': 1, 'attempt': 1, 'provid': 1, 'detail': 1, 'work': 1, 'spur': 1, 'grow': 1, 'libr': 1, 'country': 1, 'abroad': 1}
43


In [24]:
# check number of elements in qry_terms (should be 112)
print(len(qry_terms))

# check terms for first element in qry_terms
print(qry_terms.get("1"))

# check how many unique terms are in the first element of qry_terms (should be 14)
print(len(qry_terms.get("1")))

112
{'problem': 1, 'concern': 1, 'mak': 1, 'describ': 1, 'titl': 3, 'difficul': 1, 'involv': 1, 'autom': 1, 'retriev': 1, 'artic': 2, 'approxim': 1, 'us': 1, 'relev': 1, 'cont': 1}
14


## 5.2 Representing data in a shared space

To lay the groundwork for determining how relevant a document is to a given query, we need to represent our documents and queries in the vector space. First, we collect all the terms present in both the documents and queries together as our vocabulary. Then, we loop through all the documents and queries and assign each a vector. Each vector is of length same as the number of terms in our vocabulary. Each time a term appears in a document or query, that document or query's vector is updated to include that instance of that term.

In [25]:
# collect all terms from documents and queries and return as sorted list
# this will be the collective vocabulary
def collect_vocabulary():
    
    # make empty list for all terms
    all_terms = []
    
    # iterate over doc_terms dictionary
    for doc_id in doc_terms.keys():
        
        # iterate over terms in each doc_id
        for term in doc_terms.get(doc_id).keys():
            
            # append term to all_terms list
            all_terms.append(term)
    
    # repeat for queries
    for qry_id in qry_terms.keys():
        
        for term in qry_terms.get(qry_id).keys():
            
            all_terms.append(term)
            
    # return sorted list
    return sorted(set(all_terms))

# call function to collect vocabulary
all_terms = collect_vocabulary()

In [26]:
# check how many terms in vocabulary (should be 7,775)
print(len(all_terms))

# check first 10 elements in vocabulary
print(all_terms[:10])

7775
["''", "'60", "'70", "'anyhow", "'apparent", "'basic", "'better", "'bibliograph", "'bibliometrics", "'building"]


In [27]:
# define function that takes in vocabulary and returns
# dictionary 'output' that has ids as keys and vectors
# as values (list of counts that corresponds to 
# collective vocabulary)

def vectorize(input_features, vocabulary):
    
    # create empty dictionary for output
    output = {}
    
    # iterate over ids in input_features
    for item_id in input_features.keys():
        
        # save item_id's value in input_features as features
        features = input_features.get(item_id)
        
        # create empty list for vector
        output_vector = []
        
        # iterate over terms in vocabulary
        for word in vocabulary:
            
            # if word is in the features dictionary
            if word in features.keys():
                
                # append the count of the word in features to output_vector
                output_vector.append(int(features.get(word)))
                
            else:
                
                # append '0' to output_vector (word is not in item)
                output_vector.append(0)
        
        # assign output_vector as value to item_id key in output dict
        output[item_id] = output_vector
    
    return output

# call vectorize on doc_terms and qry_terms
doc_vectors = vectorize(doc_terms, all_terms)
qry_vectors = vectorize(qry_terms, all_terms)

In [28]:
# check number of doc_vectors (should be 1460)
print(len(doc_vectors))

# check vectors of doc 1460
print(doc_vectors.get("1460"))

1460
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [29]:
# check number of qry_vectors (should be 112)
print(len(qry_vectors))

# check vectors of query 112
print(qry_vectors.get("112"))

112
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## 5.3 Inverse document frequency

If there is a match between a query and a document on a relatively common word (like "the", "and", "so", etc.), we don't want that match to count as much as a match on a relatively rare word (like "cost", "dissemination", "retrieval", etc.). That is, a match on a less common word will indicate that the document is more relevant to the query than a match on a more common word.

To implement this functionality, we need to calculate the inverse document frequency (idf) for all the terms in the vocabulary, so we know which terms are more and less common. Then, we pass on those frequencies to the terms in each document vector, so the vectors contain information about term importance.

In [30]:
# define function to calculate idfs for all terms in vocabulary
def calculate_idfs(vocabulary, doc_features):
    
    # create empty dictionary for doc idfs
    doc_idfs = {}
    
    # iterate over terms in vocabulary
    for term in vocabulary:
        
        # initialize doc_count as 0
        doc_count = 0
        
        # iterate over doc_ids in doc_features
        for doc_id in doc_features.keys():
            
            # save dict of term frequency to terms
            terms = doc_features.get(doc_id)
            
            # if term is already in terms
            if term in terms.keys():
                
                # increment doc_count
                doc_count += 1
                
        # apply idf formula and save as value for term key in doc_idfs
        # +1 is to smooth out counts so we never divide by 0
        # log is to tone down differences in absolute counts
        doc_idfs[term] = math.log(float(len(doc_features.keys())) /
                                 float(1 + doc_count), 10)
        
    return doc_idfs

# call function to calculate document idfs
doc_idfs = calculate_idfs(all_terms, doc_terms)

In [31]:
# check number of terms in vocabulary (should be 7775)
print(len(doc_idfs))

# check idf value for one term ("system")
print(doc_idfs.get("system"))

7775
0.4287539560862571


In [32]:
# define function that applies idf weights to input_terms
def vectorize_idf(input_terms, input_idfs, vocabulary):
    
    # create empty dictionary for output
    output = {}
    
    # iterate over item_ids in input_terms
    for item_id in input_terms.keys():
        
        # save value as terms
        terms = input_terms.get(item_id)
        
        # create empty list for output vector
        output_vector = []
        
        # iterate over terms in vocabulary
        for term in vocabulary:
            
            # if the term is in the terms dict
            if term in terms.keys():
                
                # append to output_vector the product of term frequencies
                # and idf weights of that term
                output_vector.append(input_idfs.get(term) * 
                                     float(terms.get(term)))
            else:
                
                # append 0 to output_vector (term is not in doc)
                output_vector.append(float(0))
        
        # assign output_vector to item_id in output dictionary
        output[item_id] = output_vector
    
    return output

# call function to apply idf weights to doc_terms
doc_vectors = vectorize_idf(doc_terms, doc_idfs, all_terms)

In [33]:
# check number of doc_vectors (should be 1460)
print(len(doc_vectors))

# check vectors for one document (instead of counts
# of terms, should now have idfs * counts)
print(doc_vectors.get("1460"))

1460
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.8633228601204554, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.08

# 6. Final search algorithm

We can now put together all components from above to implement a search algorithm that will return documents in order of relevance to a query. Relevance is determined by distance between query and document vectors in the shared vector space.

## 6.1 Helper functions

To eliminate the effect of different lengths of vectors, we will look at the cosine of the angle between the vectors. Our relevance function will be:

cosine(vec1, vec2) = dot_product(vec1, vec2) / (length(vec1) * length(vec2))

In [34]:
# calculate the length of a vector (helper function)
def length(vector):
    
    # set sq_length to 0
    sq_length = 0
    
    # iterate over all elements in vector
    for index in range(0, len(vector)):
        
        # add all elements (squared) to sq_length
        sq_length += math.pow(vector[index], 2)
        
    # return sqrt of sq_length    
    return math.sqrt(sq_length)

In [35]:
# calculate the dot_product of two vectors (helper function)
def dot_product(vector1, vector2):
    
    # both vectors have to be the same length to compute dot_product
    if len(vector1) == len(vector2):
        
        # set initial dot_product to 0
        dot_prod = 0
        
        # iterate over all elements in vector length
        for index in range(0, len(vector1)):
            
            # if either number in vector at that index is 0, we don't
            # want to multiply by 0 (will take out value in other vector)
            if not vector1[index] == 0 and not vector2[index] == 0:
                
                # add the product of elements in both vectors to dot_prod
                dot_prod += vector1[index] * vector2[index]
        
        return dot_prod
                
    else: 
        
        # if vectors don't match in length
        return "Unmatching dimensionality"

In [36]:
# calculate cosine similarity between vectors (e.g., query
# vector and document vector) (helper function)
def calculate_cosine(query, document):
    
    cosine = dot_product(query, document) / (length(query) * length(document))
    
    return cosine

## 6.2 Run the algorithm

In [37]:
# select a qry_id
query = qry_vectors.get("3")

# initialize an empty dictionary to store the resulting list of documents
results = {}

# iterate over the document ids' vectors
for doc_id in doc_vectors.keys():
    
    # save vector to variable "document"
    document = doc_vectors.get(doc_id)
    
    # calculate the cosine between the query and the document's vectors
    cosine = calculate_cosine(query, document)
    
    # save cosine in results dictionary with doc_id as key
    results[doc_id] = cosine

# iterate over items (key, value pairs) in results dictionary
# pass "sorted" function: 1) the items, 2) the key to sort on (which is
# the cosine value, itemgetter(1), and 3) reverse = True to get highest first
# print first 10 results
for items in sorted(results.items(), key = itemgetter(1), reverse = True)[:10]:
    
    # print each doc_id
    print(items[0])

469
1179
1181
1142
1190
1116
445
85
599
60


# 7. Evaluation

How well does this model do compared to the gold standard? That is, how many of the top 10 or so documents our algorithm returns are also on the gold standard's list of relevant documents for any given query?

## 7.1 Precision @ k and "found"

We will treat this as a classification problem, where a true positive means that our algorithm put the document in the top 10 and the gold standard lists that document as relevant. We will take the precision score, which is the number of true positives over the total number of predictions. Since we are returning 10 documents per query, we will calculate precision at 10. 

Just one precision at 10 score, such as the precision at 10 score for query number three, does not tell us much about how good our algorithm is because there may be 50 relevant documents or just one relevant document for that particular query according to the gold standard. If there are 50, our model has a higher chance of picking a relevant document randomly than if there is only one. So instead, we will evaluate our model's performance on its mean precision at 10 score across all queries.  

In [38]:
# to speed up the algorithm, this function prefilters the documents
# and keeps only documents that contain at least one word from 
# the search query for the algorithm to go through
def prefilter(doc_terms, query):
    docs = []
    for doc_id in doc_terms.keys():
        found = False
        i = 0
        while i<len(query.keys()) and not found:
            term = list(query.keys())[i]
            if term in doc_terms.get(doc_id).keys():
                docs.append(doc_id)
                found=True
            else:
                i+=1
    return docs

In [39]:
# check on query 6
docs = prefilter(doc_terms, qry_terms.get("6"))

# should start with 5 and end with 242
print(docs[:100])

# should be 607
print(len(docs))

['5', '6', '10', '15', '16', '17', '21', '22', '25', '26', '27', '29', '30', '33', '38', '41', '42', '43', '45', '46', '47', '49', '51', '52', '56', '57', '58', '63', '64', '66', '68', '71', '74', '77', '78', '79', '80', '82', '87', '90', '91', '92', '95', '96', '97', '98', '101', '102', '104', '105', '106', '107', '109', '114', '116', '117', '122', '123', '124', '126', '129', '131', '132', '136', '140', '141', '142', '144', '150', '151', '155', '157', '158', '159', '160', '163', '168', '169', '175', '178', '179', '180', '181', '191', '194', '197', '206', '208', '211', '212', '214', '218', '220', '228', '229', '233', '237', '240', '241', '242']
607


In [40]:
# run prefiltering helper function

prefiltered_docs = {}
for query_id in mappings.keys():
    prefiltered_docs[query_id] = prefilter(doc_terms, qry_terms.get(str(query_id)))

In [41]:
# calculate precision of model against the gold standard
def calculate_precision(model_output, gold_standard):
    
    # initialize a true positives count at 0
    true_pos = 0
    
    # iterate over model output doc_ids
    for item in model_output:
        
        # if doc_id is in the gold standard list
        if item in gold_standard:
            
            # increment true positives count by 1
            true_pos += 1
    
    # return the mean precision over all model outputs
    return float(true_pos) / float(len(model_output))

In [42]:
# alternative measure of success: did the model find at least
# 1 document in the top 10 that is included in the gold
# standard list?
def calculate_found(model_output, gold_standard):
    
    # initialize count for if a relevant document is found
    found = 0
    
    # iterate over doc_ids in model output
    for item in model_output:
        
        # if the doc_id is also in the gold standard list
        if item in gold_standard:
            
            # make found count 1
            found = 1
    
    return float(found)

In [43]:
# initialize variables for precision and found
precision_all = 0.0
found_all = 0.0

# iterate over all query_ids in mappings
for query_id in mappings.keys():
    
    # get gold standard list of relevant documents
    gold_standard = mappings.get(str(query_id))
    
    # get the query vector
    query = qry_vectors.get(str(query_id))
    
    # initialize an empty dictionary to store results
    results = {}
    
    # initialize an empty list to store doc_ids that
    # the model finds most relevant
    model_output = []
    
    # iterate over doc_id in prefiltered doc vectors dict
    for doc_id in prefiltered_docs.get(str(query_id)):
        
        # get the document's vector
        document = doc_vectors.get(doc_id)
        
        # calculate the cosine between query and doc
        cosine = calculate_cosine(query, document)
        
        # store cosine in results dict with doc_id as key
        results[doc_id] = cosine
    
    # iterate over key, value pairs in sorted results dict
    # just top 10 doc_ids
    for items in sorted(results.items(),
                       key = itemgetter(1),
                       reverse = True)[:10]:
        
        # append doc_id to model_output list
        model_output.append(items[0])
    
    # calculate precision between model output and gold standard
    precision = calculate_precision(model_output, gold_standard)
    
    # calculate found
    found = calculate_found(model_output, gold_standard)
    
    # print precision and found for this query id
    print(f"Query ID {str(query_id)} precision: {str(precision)}, found: {str(found)}")
    
    # add precision for this query to model's total precision score
    precision_all += precision
    
    # add found for this query to model's total found score
    found_all += found

Query ID 1 precision: 0.9, found: 1.0
Query ID 2 precision: 0.1, found: 1.0
Query ID 3 precision: 0.8, found: 1.0
Query ID 4 precision: 0.0, found: 0.0
Query ID 5 precision: 0.1, found: 1.0
Query ID 6 precision: 0.0, found: 0.0
Query ID 7 precision: 0.0, found: 0.0
Query ID 8 precision: 0.0, found: 0.0
Query ID 9 precision: 0.5, found: 1.0
Query ID 10 precision: 0.4, found: 1.0
Query ID 11 precision: 0.3, found: 1.0
Query ID 12 precision: 0.1, found: 1.0
Query ID 13 precision: 0.5, found: 1.0
Query ID 14 precision: 0.0, found: 0.0
Query ID 15 precision: 0.1, found: 1.0
Query ID 16 precision: 0.0, found: 0.0
Query ID 17 precision: 0.1, found: 1.0
Query ID 18 precision: 0.2, found: 1.0
Query ID 19 precision: 0.3, found: 1.0
Query ID 20 precision: 0.4, found: 1.0
Query ID 21 precision: 0.0, found: 0.0
Query ID 22 precision: 0.0, found: 0.0
Query ID 23 precision: 0.2, found: 1.0
Query ID 24 precision: 0.7, found: 1.0
Query ID 25 precision: 0.1, found: 1.0
Query ID 26 precision: 0.7, found:

In [44]:
# get the model's total precision score
rounded_precision = round(precision_all/float(len(mappings.keys())), 2)
print(f"Mean precision of model: {str(rounded_precision)}")

# get the model's total found score
rounded_found = round(found_all/float(len(mappings.keys())), 2)
print(f"Mean found of model: {str(rounded_found)}")

Mean precision of model: 0.32
Mean found of model: 0.83


The mean precision score of our model, 0.32, means that on average (across all queries), only about 3 out of the 10 documents our model returned as the top 10 most relevant were on the gold standard list of relevant documents.

The mean found score of 0.83 means that the user will be able to find at least one relevant document in the top 10 results 83% of the time.

## 7.2 Top document relevance

Another way to measure the success of the search algorithm is to see if the top (= most relevant) document returned by the search algorithm is in the gold standard list (this is the same as precision @ 1). The code changes slightly from the above.

In [45]:
precision_all = 0.0
for query_id in mappings.keys():
    gold_standard = mappings.get(str(query_id))
    query = qry_vectors.get(str(query_id))
    result = ""
    model_output = []
    max_sim = 0.0
    prefiltered_docs = prefilter(doc_terms, qry_terms.get(str(query_id)))
    for doc_id in prefiltered_docs:
        document = doc_vectors.get(doc_id)
        cosine = calculate_cosine(query, document) 
        if cosine >= max_sim:
            max_sim = cosine
            result = doc_id
    model_output.append(result)
    precision = calculate_precision(model_output, gold_standard)
    print(f"{str(query_id)}: {str(precision)}")
    precision_all += precision

1: 1.0
2: 0.0
3: 1.0
4: 0.0
5: 0.0
6: 0.0
7: 0.0
8: 0.0
9: 0.0
10: 1.0
11: 1.0
12: 0.0
13: 1.0
14: 0.0
15: 0.0
16: 0.0
17: 0.0
18: 0.0
19: 0.0
20: 0.0
21: 0.0
22: 0.0
23: 0.0
24: 1.0
25: 0.0
26: 1.0
27: 1.0
28: 1.0
29: 1.0
30: 1.0
31: 0.0
32: 0.0
33: 0.0
34: 1.0
35: 0.0
37: 1.0
39: 1.0
41: 1.0
42: 1.0
43: 0.0
44: 0.0
45: 1.0
46: 0.0
49: 0.0
50: 1.0
52: 1.0
54: 0.0
55: 1.0
56: 1.0
57: 0.0
58: 1.0
61: 0.0
62: 1.0
65: 1.0
66: 1.0
67: 0.0
69: 1.0
71: 0.0
76: 1.0
79: 1.0
81: 1.0
82: 0.0
84: 0.0
90: 0.0
92: 1.0
95: 0.0
96: 0.0
97: 1.0
98: 1.0
99: 0.0
100: 0.0
101: 0.0
102: 1.0
104: 0.0
109: 0.0
111: 1.0


In [46]:
print(precision_all/len(mappings.keys()))

0.4473684210526316


This score of approximately 0.45 means that the first document returned by our search algorithm is a relevent document according to the gold standard for 45% of the queries.

## 7.3 Mean reciprocal rank

Yet another way to measure the results of our algorithm is to see which returned document to any query is the first relevant document for that query, according to the gold standard. This is called rank; for example, if the first document returned by our algorithm is also on the gold standard list, it gets rank 1; if the second document is the first one on the gold standard list (the first returned document was not on the list), it gets rank 2, and so on.

Because rank 1 is the best possible result, it should be assigned a score of 1. To make higher ranks have a lower score (because they are worse results), each rank will be assigned a score of one over the rank number, i.e., the reciprocal rank.

To evaluate the model on all queries, we will look at the mean reciprocal rank.

In [47]:
rank_all = 0.0
for query_id in mappings.keys():
    gold_standard = mappings.get(str(query_id))
    query = qry_vectors.get(str(query_id))
    results = {}
    for doc_id in doc_vectors.keys():
        document = doc_vectors.get(doc_id)
        cosine = calculate_cosine(query, document)    
        results[doc_id] = cosine
    sorted_results = sorted(results.items(), key=itemgetter(1), reverse=True)
    index = 0
    found = False
    while found==False:
        item = sorted_results[index]
        index += 1
        if index==len(sorted_results):
            found = True
        if item[0] in gold_standard:
            found = True
            print(f"{str(query_id)}: {str(float(1) / float(index))}")
            rank_all += float(1) / float(index)

1: 1.0
2: 0.3333333333333333
3: 1.0
4: 0.09090909090909091
5: 0.14285714285714285
6: 0.038461538461538464
7: 0.043478260869565216
8: 0.02857142857142857
9: 0.5
10: 1.0
11: 1.0
12: 0.1
13: 1.0
14: 0.011494252873563218
15: 0.125
16: 0.029411764705882353
17: 0.25
18: 0.25
19: 0.25
20: 0.5
21: 0.05555555555555555
22: 0.09090909090909091
23: 0.5
24: 1.0
25: 0.1111111111111111
26: 1.0
27: 1.0
28: 1.0
29: 1.0
30: 1.0
31: 0.5
32: 0.3333333333333333
33: 0.05555555555555555
34: 1.0
35: 0.5
37: 1.0
39: 1.0
41: 1.0
42: 1.0
43: 0.14285714285714285
44: 0.5
45: 1.0
46: 0.5
49: 0.3333333333333333
50: 1.0
52: 1.0
54: 0.3333333333333333
55: 1.0
56: 1.0
57: 0.09090909090909091
58: 1.0
61: 0.3333333333333333
62: 1.0
65: 1.0
66: 1.0
67: 0.25
69: 1.0
71: 0.25
76: 1.0
79: 1.0
81: 1.0
82: 0.5
84: 0.05
90: 0.2
92: 1.0
95: 0.5
96: 0.08333333333333333
97: 1.0
98: 1.0
99: 0.3333333333333333
100: 0.1
101: 0.020833333333333332
102: 1.0
104: 0.25
109: 0.5
111: 1.0


In [48]:
print(rank_all/float(len(mappings.keys())))

0.5804111538527951


The result of the mean reciprocal ranking score of about 0.58 means that the first relevant document that our algorithm finds is usually between the first and second document, but more often probably the second document (reciprocal rank of 1/2 or 0.5). 

# 8. Next Steps

Apply this algorithm to your own data or another test set such as one from http://ir.dcs.gla.ac.uk/resources/test_collections/ (the Cranfield dataset is suggested). 